In [0]:
df_patient_hosa = spark.read.parquet('/mnt/bronze/hosa/patients')
df_patient_hosa.createOrReplaceTempView("patient_hosa")

df_patient_hosb = spark.read.parquet('/mnt/bronze/hosb/patients')
df_patient_hosb.createOrReplaceTempView("patient_hosb")

In [0]:
%sql
SELECT * FROM patient_hosa

In [0]:
%sql
SELECT * FROM patient_hosb

In [0]:
%sql
-- implementing cdm - Common Data Model !!!! Because the col name for both hosa and hosb patient is different. 

CREATE OR REPLACE TEMP VIEW cdm_patients AS 
SELECT CONCAT(SRC_PatientID,'-',datasource) As Patient_Key, *
FROM (
  SELECT 
  PatientID AS SRC_PatientID ,
  FirstName,
  LastName,
  MiddleName,
  SSN,
  PhoneNumber,
  Gender,
  DOB,
  Address,
  ModifiedDate,
  datasource
  FROM patient_hosa
  UNION ALL
  SELECT 
  ID AS SRC_PatientID,
  F_Name AS FirstName,
  L_Name AS LastName,
  M_Name ASMiddleName,
  SSN,
  PhoneNumber,
  Gender,
  DOB,
  Address,
  Updated_Date AS ModifiedDate,
  datasource
  FROM patient_hosb
)


In [0]:
%sql

SELECT * FROM cdm_patients

In [0]:
%sql
-- Implementing quality check and moving the bad data to isQuarentined

CREATE OR REPLACE TEMP VIEW quality_checks AS
SELECT 
  Patient_Key,
  SRC_PatientID,
  FirstName,
  LastName,
  MiddleName,
  SSN,
  PhoneNumber,
  Gender,
  DOB,
  Address,
  ModifiedDate As SRC_ModifiedDate,
  datasource,
  CASE 
    WHEN SRC_PatientID IS NULL OR dob IS NULL OR firstname IS NULL or lower(firstname)='null' THEN TRUE
    ELSE FALSE
    END AS is_bad_data
FROM cdm_patients


In [0]:
%sql
select * from quality_checks
order by is_bad_data desc

In [0]:

%sql
CREATE TABLE IF NOT EXISTS silver.patients (
    Patient_Key STRING,
    SRC_PatientID STRING,
    FirstName STRING,
    LastName STRING,
    MiddleName STRING,
    SSN STRING,
    PhoneNumber STRING,
    Gender STRING,
    DOB DATE,
    Address STRING,
    SRC_ModifiedDate TIMESTAMP,
    datasource STRING,
    is_bad_data BOOLEAN,
    inserted_date TIMESTAMP,
    modified_date TIMESTAMP,
    is_current BOOLEAN
)
USING DELTA;


In [0]:
%sql 

-- Now we will implement SCD 2 - We will take use of the cols - insert_date, modify_date, is_current
-- Insert_date will store the date when the row data was inserted 
-- Modified_date will store the date when the row was modified, this will same as insert_timestamp, when the record first comes in to the table
-- is_current - boolean - this will hold true/false value if the data is current data then true, if previous data then false

--STEP1
-- Here first we will mark the existing record as historical record if a same record of same key comes in !!!

MERGE INTO silver.patients AS target 
USING quality_checks AS source
ON target.Patient_Key = source.Patient_Key
AND target.is_current = true 
WHEN MATCHED
AND (
  target.SRC_PatientID <> source.SRC_PatientID OR
  target.FirstName <> source.FirstName OR
  target.LastName <> source.LastName OR
  target.MiddleName <> source.MiddleName OR
  target.SSN <> source.SSN OR
  target.PhoneNumber <> source.PhoneNumber OR
  target.Gender <> source.Gender OR
  target.DOB <> source.DOB OR
  target.Address <> source.Address OR
  target.SRC_ModifiedDate <> source.SRC_ModifiedDate OR
  target.datasource <> source.datasource OR
  target.is_bad_data <> source.is_bad_data
)
THEN UPDATE SET 
  target.is_current=false,
  target.modified_date = current_timestamp()

WHEN NOT MATCHED
THEN INSERT(
  Patient_Key,
  SRC_PatientID,
  FirstName,
  LastName,
  MiddleName,
  SSN,
  PhoneNumber,
  Gender,
  DOB,
  Address,
  SRC_ModifiedDate,
  datasource,
  is_bad_data,
  inserted_date,
  modified_date,
  is_current
)VALUES(
  source.Patient_Key,
  source.SRC_PatientID,
  source.FirstName,
  source.LastName,
  source.MiddleName,
  source.SSN,
  source.PhoneNumber,
  source.Gender,
  source.DOB,
  source.Address,
  source.SRC_ModifiedDate,
  source.datasource,
  source.is_bad_data,
  current_timestamp(),
  current_timestamp(),
  true
  )



In [0]:
%sql
--STEP 2: The records that came again, now these should be uploaded as new records with current flag as true

MERGE INTO silver.patients AS target
USING quality_checks AS source
ON target.Patient_Key = source.Patient_Key
AND target.is_current= true -- THIS  WILL MAKE THEM NOT MATCHED, AS WE MADE THE EARLIER RECORD FALSE
WHEN NOT MATCHED
THEN INSERT(
  Patient_Key,
  SRC_PatientID,
  FirstName,
  LastName,
  MiddleName,
  SSN,
  PhoneNumber,
  Gender,
  DOB,
  Address,
  SRC_ModifiedDate,
  datasource,
  is_bad_data,
  inserted_date,
  modified_date,
  is_current
)
VALUES(
  source.Patient_Key,
  source.SRC_PatientID,
  source.FirstName,
  source.LastName,
  source.MiddleName,
  source.SSN,
  source.PhoneNumber,
  source.Gender,
  source.DOB,
  source.Address,
  source.SRC_ModifiedDate,
  source.datasource,
  source.is_bad_data,
  current_timestamp(), -- Set inserted_date to current timestamp
  current_timestamp(), -- Set modified_date to current timestamp
  true -- Mark as current
 );


In [0]:
%sql
select count(*),Patient_Key from silver.patients
group by patient_key
order by 1 desc